# Arup Festival of Data

## Welcome

Here you will find code used to build synthetic city data for the Arup Festival of Data challenge.

## What is City Chef?

We've been researching and building new approaches to modelling human behaviour in cities and countries. With the aim of helping design better and farer policies and infrastructure for the future. A hugely important consideration of our work is the complexity and hetrogeneity of humans. No one is the same and no one should be treated the same when trying to plan the future.

But this often puts us in an awkward possition for our research - data about people and their behaiour is useful to us, but accessing it is hard, the quality is often dubious and we prefer not to work with sensitive personal data.

City Chef is our tool for working around this data challenge. City Chef builds fake data for use by researchers and modellers looking to develop and test new approaches. City Chef has some key aims:

1. Output data that is often not obsevred or measured
2. Output data in useful formats
3. Use representative physical components, such as networks
4. Use representative probabilistic components, such as age distributions

and where (4) fails: (5) Use representative complexity in the distributions

## Outputs

Our focus is often to produce human level data, such as **household census**, **commuter surveys** and **household travel surveys**. But we also create the following city data:

**Facilities:** activity locations (including households)

**Road Networks:**

**Road Transit Routes:** Car routes using the road network

**Rail Transit Network and Routes:**

**Statistical Zones:**

**Population:** Generate a population of agents, with consistent household attributes. The distribution of the population attributes is dependant on the above spatial features and some encoded underlying distributions.

**Plans:** Generate simple activity based plans for each agent, currently only supports simple tour based plans. The distribution of the plan attributes, such as activity choice and mode, is dependant on the agent, facility and network features.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import networkx as nx
from scipy.spatial.distance import cdist
import os

from citychef import spatial
from citychef import graph
from citychef import household as hh
from citychef import person
from citychef import tree
from citychef import choice
from citychef import osm, gtfs
from citychef.household import minmax

ModuleNotFoundError: No module named 'citychef'

In [ ]:
city_dir = "./city_A"

In [ ]:
if not os.path.exists(city_dir):
    os.mkdir(city_dir)
else:
    print("WARNING - potentially overwritting previous results")

In [ ]:
# city A
TARGET_HHS = 2000
SEED_AREA = [[275000,780000],[277000,782000]]
TARGET_CENTRES = 5
BUS_ROUTES = 12
TRAIN_ROUTES = 4

# city B
# TARGET_HHS = 2000
# SEED_AREA = [[275000,780000],[277000,782000]]
# TARGET_CENTRES = 4
# BUS_ROUTES = 12
# TRAIN_ROUTES = 4

In [ ]:
(x0, y0), (x1, y1) = SEED_AREA
area = (x1-x0) * (y1-y0)
centres = spatial.Centres(SEED_AREA, density = TARGET_CENTRES/area)

spread = ((x1-x0) + (y1-y1))/10
facilities = {
    'households': spatial.Clusters(centres, size=TARGET_HHS, sigma=spread).crop_to_bbox(SEED_AREA),
    'work': spatial.Clusters(centres, size=TARGET_HHS/10, sigma=spread/2).crop_to_bbox(SEED_AREA),
    'leisure': spatial.Clusters(centres, size=TARGET_HHS/20, sigma=spread/2).crop_to_bbox(SEED_AREA),
    'education': spatial.Clusters(centres, size=TARGET_HHS/20, sigma=spread/3).crop_to_bbox(SEED_AREA),
    'health': spatial.Clusters(centres, size=TARGET_HHS/50, sigma=spread/3).crop_to_bbox(SEED_AREA),
    'shopping': spatial.Clusters(centres, size=TARGET_HHS/20, sigma=spread/4).crop_to_bbox(SEED_AREA),
}
bbox = spatial.collect_bbox(facilities) # adjust the bbox for max extends

spatial.write_buildings_geojson(
    facilities,
    path=os.path.join(city_dir, "building_locations.geojson"),
    epsg="epsg:27700",
    to_epsg="epsg:4326"
    )

In [ ]:
car_network = graph.TreeNetwork(bbox, facilities['households'], grid='regular', max_points=50, label="highway")

# osm.nx_to_osm(
#     g=car_network.g,
#     path=os.path.join(city_dir, "car_osm.xml")
# )
graph.nx_to_geojson(
    g=car_network.g,
    path=os.path.join(city_dir, "car_network.geojson"),
    epsg="epsg:27700",
    to_epsg="epsg:4326"
)
# fig.savefig(os.path.join(city_dir, 'road_network.png'))

In [ ]:
buses = graph.Transit(car_network, facilities['households'], density_radius=1000)
bus_routes = buses.build_routes(num_routes=BUS_ROUTES, max_length=30, min_length=10, straightness=2)
total_bus_graph = buses.graph

gtfs.build_gtfs(
    transit=buses,
    name="bus",
    out_dir=city_dir,
    agency_id=0,
    agency_name='bus_connect',
    frequency=10,
    from_epsg="epsg:27700",
    to_epsg="epsg:4326"
)

graph.nx_to_geojson(
    g=buses.graph,
    path=os.path.join(city_dir, "bus_network.geojson"),
    epsg="epsg:27700",
    to_epsg="epsg:4326"
)

In [ ]:
potential_stations = np.array([v['pos'] for k, v in car_network.g.nodes.items() if k[:2] == '00'])
rail_network = graph.DelaunayNetwork(potential_stations)

trains = graph.Transit(rail_network, facilities['households'])
train_routes = trains.build_routes(num_routes=TRAIN_ROUTES, max_length=12, min_length=10, straightness=20)

total_train_graph = trains.graph

gtfs.build_gtfs(
    transit=trains,
    name="rail",
    out_dir=city_dir,
    agency_id=1,
    agency_name='rail_get_you_there',
    frequency=30,
    from_epsg="epsg:27700",
    to_epsg="epsg:4326"
)

graph.nx_to_geojson(
    g=trains.graph,
    path=os.path.join(city_dir, "rail_network.geojson"),
    epsg="epsg:27700",
    to_epsg="epsg:4326"
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
spatial.plot_facilities(facilities, centres=centres, ax=ax)
car_network.plot(ax=ax)
buses.plot(ax=ax)
trains.plot(ax=ax, line_colour='blue')
fig.patch.set_visible(False)
# ax.axis('off')
# plt.axis('equal')

# fig.savefig(os.path.join(city_dir, 'city.png'))

In [ ]:
hh_df = pd.DataFrame(range(facilities['households'].size), columns=['hh_index'])
hh_hidden_df = pd.DataFrame(range(facilities['households'].size), columns=['hh_index'])

In [ ]:
zones = tree.Zones(
    bbox=bbox, 
    facilities=facilities['households'],
    max_zone_facilities=400,
    max_sub_zone_facilities=100,
)
zones.plot()

zones.zone_gdf.crs = "EPSG:27700"
zones.zone_gdf.to_crs("epsg:4226").to_file(os.path.join(city_dir, "administrative_areas.geojson"), driver='GeoJSON')

zones.sub_zone_gdf.crs = "EPSG:27700"
zones.sub_zone_gdf.to_crs("epsg:4226").to_file(os.path.join(city_dir, "administrative_zones.geojson"), driver='GeoJSON')


In [ ]:
# fig.savefig(os.path.join(city_dir, 'density.png'))

In [ ]:
hhs = facilities['households']
work = facilities['work']
leisure = facilities['leisure']
education = facilities['education']
health = facilities['health']
shopping = facilities['shopping']

hh_dist_to_centre = hhs.dist_to_centres()
hh_dist_to_centre_mm = minmax(hh_dist_to_centre)

hh_df['area_id'] = zones.facility_zone_ids
hh_df['zone_id'] = zones.facility_sub_zone_ids

n = centres.size

hh_hidden_df['density_mm'] = spatial.density(hhs, hhs)
hh_hidden_df['dist_closest_centre_mm'] = spatial.distances_to_closest(hhs, centres, 1)
hh_hidden_df['dist_closest_centres_mm'] = spatial.distances_to_closest(hhs, centres, max([n,3]))
hh_hidden_df['density_work_places_mm'] = spatial.density(hhs, work, 500)
hh_hidden_df['density_leisure_mm'] = spatial.density(hhs, leisure, 500)
hh_hidden_df['dist_closest_schools_mm'] = spatial.distances_to_closest(hhs, education, max([n,2]))
hh_hidden_df['dist_closest_health_mm'] = spatial.density(hhs, health, 500)

hh_locs_mm = minmax(hhs.locs)
hh_centre_ids_mm = minmax(hhs.ids)

hh_hidden_df['hidden'] = hh.gen_hidden(hh_locs_mm[:,0], hh_locs_mm[:,1], hh_hidden_df['density_mm'])

In [ ]:
hh_hidden_df.head(10)

### Household Features

Household features are generated based on hidden features and previously generated features. All generative funtions combine some level of probabalitic sampling. Data sampling is designed to generate non linear and complex dependancies. The aim of which is to create a population with complex and noisy interdependencies between features.

In [ ]:
hh_df['hh_count'] = hh.gen_hh_count(
    hh_hidden_df['hidden'], 
    hh_hidden_df['density_mm']
)

hh_df['hh_children'] = hh.gen_num_children(
    hh_df['hh_count'],
    hh_hidden_df['hidden'],
    hh_hidden_df['dist_closest_schools_mm'],
    hh_hidden_df['density_leisure_mm']
)

hh_hidden_df['age_group'] = hh.gen_age_group(
    hh_df['hh_children'],
    hh_hidden_df['hidden'],
    hh_hidden_df['density_mm']
)

hh_df['hh_people_in_work'] = hh.get_people_in_work(
    hh_hidden_df['age_group'],
    hh_hidden_df['hidden'],
    hh_df['hh_count'],
    hh_df['hh_children'],
    hh_hidden_df['density_mm']
)

hh_df['hh_in_work'] = hh.get_in_work(
    hh_df['hh_people_in_work']
)

hh_df['hh_income'] = hh.get_income(
    hh_df['hh_in_work'],
    hh_df['hh_count'],
    hh_df['hh_children'],
    hh_hidden_df['density_work_places_mm'],
    hh_hidden_df['density_mm']
)

hh_income_mm = minmax(hh_df['hh_income'])

hh_df['hh_cars'] = hh.get_cars(
    hh_hidden_df['hidden'],
    hh_income_mm,
    hh_hidden_df['density_mm'],
    hh_df['hh_count'],
    hh_df['hh_children']
)

In [ ]:
hh_df.head()

### Individual Features

Individuals features are generated based on their household hidden features and previously generated features. As per for households, all generative funtions combine some level of probabalitic sampling. Data sampling is designed to generate non linear and complex dependancies. The aim of which is to create a population with complex and noisy interdependencies between features.

In [ ]:
# hh features
hh_counts = hh_df['hh_count'].to_numpy()
agent_hh_array = np.repeat(hh_df.to_numpy(), hh_counts, axis=0)
person_df = pd.DataFrame(agent_hh_array)
person_df.columns = hh_df.columns
person_df['p_hh_index'] = np.array([i for c in hh_df['hh_count'] for i in range(c)])

#hidden vectors
agent_hidden_array = np.repeat(hh_hidden_df.to_numpy(), hh_counts, axis=0)
person_hidden_df = pd.DataFrame(agent_hidden_array)
person_hidden_df.columns = hh_hidden_df.columns
person_hidden_df['p_hh_index'] = np.array([i for c in hh_df['hh_count'] for i in range(c)])

In [ ]:
person_df['adult'] = person.get_is_adult(
    person_df['p_hh_index'],
    person_df['hh_count'],
    person_df['hh_children']
)

person_df['gender'] = person.get_gender(
    person_df['p_hh_index'], 
    person_df['adult'],
    person_df['hh_children'],
    person_hidden_df['hidden']
)

person_df['age'] = person.get_age(
    person_hidden_df['hidden'],
    person_hidden_df['age_group'],
    person_df['adult']
)

person_df['employment'] = person.employment(
    person_df['adult'],
    person_df['hh_people_in_work'],
    person_df['p_hh_index'],
    person_df['age'],
    person_hidden_df['hidden'],
    person_hidden_df['dist_closest_schools_mm'],
    person_hidden_df['density_work_places_mm'],
    person_df['hh_income'],
    person_hidden_df['density_mm'],
)

person_df['occupation'] = person.occupation(
    person_df['employment'],
    person_df['age'],
    minmax(person_df['hh_income']),
)

person_df.head(10)

In [ ]:
person_hidden_df.head(10)

In [ ]:
print(len(person_df))
print(len(hh_df))

### Activity Choice

A simple activity plan is generated based on individual features. This plan consists of a single activity at a given facility location.

#### Method
1. Activity type choice is made based on household and individual features. This can include staying at home.
2. Facilities for each activity type are weighted by their desirability (based on density and distance).
3. Agents randomly choose a facility based on this weighting and on their individual features.

In [ ]:
# Main activity choice
person_df['main_activity'] = choice.main_activity_choice(
    person_df['employment'],
    person_hidden_df['density_work_places_mm'],
    person_df['occupation'],
    person_hidden_df['hidden'],
    minmax(person_df['hh_income']),
    person_df['hh_count'],
    person_hidden_df['dist_closest_centres_mm'],
    person_hidden_df['density_leisure_mm'],
    person_df['age'],
)

In [ ]:
person_df.head()

In [ ]:
# FACILITY FEATURES
zone_gdf = zones.sub_zone_gdf

work_df = pd.DataFrame(range(work.size), columns=['index'])
work_df['density_mm'] = spatial.density(work, work, 500)
work_df['geometry'] = [Point(x,y) for x,y in work.locs]
work_gdf = gpd.GeoDataFrame(work_df, geometry='geometry')
work_gdf = gpd.sjoin(work_gdf, zone_gdf, how="left", op='within').drop('index_right', axis=1)

leisure_df = pd.DataFrame(range(leisure.size), columns=['index'])
leisure_df['density_mm'] = spatial.density(leisure, shopping, 500)
leisure_df['geometry'] = [Point(x,y) for x,y in leisure.locs]
leisure_gdf = gpd.GeoDataFrame(leisure_df, geometry='geometry')
leisure_gdf = gpd.sjoin(leisure_gdf, zone_gdf, how="left", op='within').drop('index_right', axis=1)

education_df = pd.DataFrame(range(education.size), columns=['index'])
education_df['density_mm'] = spatial.density(education, shopping, 500)
education_df['geometry'] = [Point(x,y) for x,y in education.locs]
education_gdf = gpd.GeoDataFrame(education_df, geometry='geometry')
education_gdf = gpd.sjoin(education_gdf, zone_gdf, how="left", op='within').drop('index_right', axis=1)

health_df = pd.DataFrame(range(health.size), columns=['index'])
health_df['density_mm'] = spatial.density(health, shopping, 500)
health_df['geometry'] = [Point(x,y) for x,y in health.locs]
health_gdf = gpd.GeoDataFrame(health_df, geometry='geometry')
health_gdf = gpd.sjoin(health_gdf, zone_gdf, how="left", op='within').drop('index_right', axis=1)

shopping_df = pd.DataFrame(range(shopping.size), columns=['index'])
shopping_df['density_mm'] = spatial.density(shopping, shopping, 300)
shopping_df['geometry'] = [Point(x,y) for x,y in shopping.locs]
shopping_gdf = gpd.GeoDataFrame(shopping_df, geometry='geometry')
shopping_gdf = gpd.sjoin(shopping_gdf, zone_gdf, how="left", op='within').drop('index_right', axis=1)


In [ ]:
hhs_work_distances = minmax(cdist(hhs.locs, work.locs))
hhs_education_distances = minmax(cdist(hhs.locs, education.locs))
hhs_shopping_distances = minmax(cdist(hhs.locs, shopping.locs))
hhs_leisure_distances = minmax(cdist(hhs.locs, leisure.locs))
hhs_health_distances = minmax(cdist(hhs.locs, health.locs))

In [ ]:
main_facility_id = np.zeros((len(person_df)))
main_facility_area = np.zeros((len(person_df)))
main_facility_zone = np.zeros((len(person_df)))

for i, (hh_index, main_act, income_mm) in enumerate(zip(person_df.hh_index, person_df.main_activity, minmax(person_df['hh_income']))):
    
    if main_act == 0:  # home
        main_facility_id[i] = -1
        main_facility_area[i] = -1
        main_facility_zone[i] = -1
        
    hh_index = int(hh_index)
        
    if main_act == 1:  # work
        facility_distances = hhs_work_distances[hh_index]
        facility_scores = (1 - income_mm) * (1 - facility_distances) + income_mm * work_df.density_mm
        facility_scores = facility_scores / facility_scores.sum()
        facitlity_id = np.random.choice(range(len(facility_scores)), p=facility_scores)
        main_facility_id[i] = facitlity_id
        main_facility_area[i], main_facility_zone[i] = work_gdf[['area_id', 'zone_id']].iloc[facitlity_id]
        continue
        
    if main_act == 2:  # education
        facility_distances = hhs_education_distances[hh_index]
        facility_scores = (1 - facility_distances) ** 2
        facility_scores = facility_scores / facility_scores.sum()
        facitlity_id = np.random.choice(range(len(facility_scores)), p=facility_scores)
        main_facility_id[i] = facitlity_id
        main_facility_area[i], main_facility_zone[i] = education_gdf[['area_id', 'zone_id']].iloc[facitlity_id]
        continue
        
    if main_act == 3:  # shopping
        facility_distances = hhs_shopping_distances[hh_index]
        facility_scores = (1 - income_mm) * (1 - facility_distances) + income_mm * shopping_df.density_mm
        facility_scores = facility_scores / facility_scores.sum()
        facitlity_id = np.random.choice(range(len(facility_scores)), p=facility_scores)
        main_facility_id[i] = facitlity_id
        main_facility_area[i], main_facility_zone[i] = shopping_gdf[['area_id', 'zone_id']].iloc[facitlity_id]
        continue
        
    if main_act == 4:  # leisure
        facility_distances = hhs_leisure_distances[hh_index]
        facility_scores = (1 - facility_distances)
        facility_scores = facility_scores / facility_scores.sum()
        facitlity_id = np.random.choice(range(len(facility_scores)), p=facility_scores)
        main_facility_id[i] = facitlity_id
        main_facility_area[i], main_facility_zone[i] = leisure_gdf[['area_id', 'zone_id']].iloc[facitlity_id]
        continue
        
    if main_act == 5:  # health
        facility_distances = hhs_health_distances[hh_index]
        facility_scores = (1 - facility_distances) ** 4
        facility_scores = facility_scores / facility_scores.sum()
        facitlity_id = np.random.choice(range(len(facility_scores)), p=facility_scores)
        main_facility_id[i] = facitlity_id
        main_facility_area[i], main_facility_zone[i] = health_gdf[['area_id', 'zone_id']].iloc[facitlity_id]
        continue

person_df['main_activity_id'] = main_facility_id
person_df['main_facility_area'] = main_facility_area
person_df['main_facility_zone'] = main_facility_zone


In [ ]:
# HH CAR CONNECTIVITY
car_ods = graph.NodesOD(car_network.g)

hh_df['car_dist'], hh_df['car_node'] = spatial.distance_index_nearest_node(hhs, car_network.locs)
work_df['car_dist'], work_df['car_node'] = spatial.distance_index_nearest_node(work, car_network.locs)
leisure_df['car_dist'], leisure_df['car_node'] = spatial.distance_index_nearest_node(leisure, car_network.locs)
education_df['car_dist'], education_df['car_node'] = spatial.distance_index_nearest_node(education, car_network.locs)
health_df['car_dist'], health_df['car_node'] = spatial.distance_index_nearest_node(health, car_network.locs)
shopping_df['car_dist'], shopping_df['car_node'] = spatial.distance_index_nearest_node(shopping, car_network.locs)

In [ ]:
hh_df.head()

In [ ]:
# HH PT CONNECTIVITY
pt_network = nx.compose(total_bus_graph, total_train_graph)
pt_pos = {n:d['pos'] for n,d in pt_network.nodes(data=True)}
pt_node_locs = np.array([d['pos'] for n,d in pt_network.nodes(data=True)])
pt_ods = graph.NodesOD(pt_network)

In [ ]:
hh_df['pt_dist'], hh_df['pt_node'] = spatial.distance_index_nearest_node(hhs, pt_node_locs)
work_df['pt_dist'], work_df['pt_node'] = spatial.distance_index_nearest_node(work, pt_node_locs)
leisure_df['pt_dist'], leisure_df['pt_node'] = spatial.distance_index_nearest_node(leisure, pt_node_locs)
education_df['pt_dist'], education_df['pt_node'] = spatial.distance_index_nearest_node(education, pt_node_locs)
health_df['pt_dist'], health_df['pt_node'] = spatial.distance_index_nearest_node(health, pt_node_locs)
shopping_df['pt_dist'], shopping_df['pt_node'] = spatial.distance_index_nearest_node(shopping, pt_node_locs)

In [ ]:
hh_df.head()

In [ ]:
car_time = np.zeros((len(person_df)))
pt_time = np.zeros((len(person_df)))
cycle_time = np.zeros((len(person_df)))
walk_time = np.zeros((len(person_df)))
mode_choice = np.zeros((len(person_df)))
journey_time = np.zeros((len(person_df)))

for i, (hh_index, main_act, main_id, income_mm, age_mm, adult, children, cars) in enumerate(
    zip(
        person_df.hh_index,
        person_df.main_activity,
        person_df.main_activity_id,
        minmax(person_df.hh_income),
        minmax(person_df.age),
        person_df.adult,
        person_df.hh_children,
        person_df.hh_cars,
              )):
    
    if main_act == 0:  # home
        mode_choice[i] = -1
        continue
        
    main_id = int(main_id)
    hh_index = int(hh_index)
    
    hh_loc = hhs.locs[hh_index]
    
    hh_car_node_dist, hh_car_node = hh_df.car_dist[hh_index], hh_df.car_node[hh_index]
    hh_car_node_time = 1.4 * 3600 * hh_car_node_dist / 25
    
    hh_pt_node_dist, hh_pt_node = hh_df.pt_dist[hh_index], hh_df.pt_node[hh_index]
    hh_pt_node_time = 3600 * hh_car_node_dist / 5
    
    times = {}
    options = {}
        
    if main_act == 1:  # work
        
        facility_loc = work.locs[main_id]
        facility_df = work_df
        
    if main_act == 2:  # education
        
        facility_loc = education.locs[main_id]
        facility_df = education_df
        
    if main_act == 3:  # shopping
        
        facility_loc = shopping.locs[main_id]
        facility_df = shopping_df
        
    if main_act == 4:  # leisure
        
        facility_loc = leisure.locs[main_id]
        facility_df = leisure_df
        
    if main_act == 5:  # health
        
        facility_loc = health.locs[main_id]
        facility_df = health_df
     
    # choice
    eucl_distance = ((hh_loc - facility_loc)**2).sum()**.5
    manh_distance = np.abs(hh_loc - facility_loc).sum()
    
    # car
    facility_car_node_dist, facility_car_node = facility_df.car_dist[main_id], facility_df.car_node[main_id]
    car_network_travel = car_ods.get(hh_car_node, facility_car_node)
    if car_network_travel != -1:
        facility_car_node_time = 1.4 * 3600 * facility_car_node_dist / 25
        car_journey_time = hh_car_node_time + facility_car_node_time + car_network_travel + 300
        if car_journey_time > 300:
            car_time[i] = car_journey_time
            times[0] = car_journey_time

            if not cars:
                options[0] = car_journey_time * 3
            else:
                options[0] = car_journey_time * (1.1 - cars / 10) * (1 - children / 10)
    
    # pt
    facility_pt_node_dist, facility_pt_node = facility_df.pt_dist[main_id], facility_df.pt_node[main_id]
    pt_network_travel = pt_ods.get(hh_pt_node, facility_pt_node)
    if pt_network_travel != -1:
        facility_pt_node_time = 3600 * facility_pt_node_dist / 5
        pt_journey_time = hh_pt_node_time + facility_pt_node_time + pt_network_travel + 120
        if pt_journey_time > 300:
            pt_time[i] = pt_journey_time
            times[1] = pt_journey_time

            options[1] = pt_journey_time * (1 + age_mm / 100) * (1 + income_mm / 10)
    
    # cycle
    cycle_journey_time = 360 * manh_distance / 15 + 120
    if (cycle_journey_time < 3600) and (cycle_journey_time > 120):
        cycle_time[i] = cycle_journey_time
        times[2] = cycle_journey_time
        
        if main_act == 3:
            options[2] = cycle_journey_time * 3
        else:
            options[2] = cycle_journey_time * (1 + age_mm / 10) * (1 + income_mm / 10)

    # walk
    walk_journey_time = 360 * eucl_distance / 5
    if walk_journey_time < 3600:
        walk_time[i] = walk_journey_time
        times[3] = walk_journey_time
        options[3] = walk_journey_time
    
    weights = np.array(list(options.values()))
    weights = weights / weights.sum()
    mode = np.random.choice(list(options), p=weights)
    time = times[mode]
    
    mode_choice[i] = mode
    journey_time[i] = time


In [ ]:
person_df['mode'] = mode_choice
person_hidden_df['journey_time'] = journey_time

In [ ]:
header = {
    'hh_in_work': {
        0: 'noone',
        1: 'yes',
    },
    'adult': {
        0: 'under16',
        1: '16+',
    },
    'gender': {
        0: 'male',
        1: 'female',
        2: 'other',
    },
    'employment': {
        0: 'retired',
        1: 'unemployed',
        2: 'education',
        3: 'employment',
    },
    'occupation': {
        0: 'none',
        1: 'unskilled',
        2: 'skilled',
        3: 'manager',
    },
    'main_activity': {
        0: 'None',
        1: 'work',
        2: 'education',
        3: 'shopping',
        4: 'leisure',
        5: 'health',
    },
    'mode': {
        0: 'car',
        1: 'PT',
        2: 'cycle',
        3: 'walk',
    },
    'hh_income_bin': {
        0: 'low',
        1: 'mid',
        2: 'mid-high',
        3: 'high',
    },
    'age_bin': {
        0: '-4',
        4: '4-13',
        13: '13-16',
        16: '16-21',
        21: '21-31',
        31: '31-41',
        41: '41-51',
        51: '51-61',
        61: '61-71',
        71: '71-81',
        81: '81+',
    }
}

In [ ]:
# import json
# with open(os.path.join(city_dir, 'header.json'), 'w') as f:
#     json.dump(header, f)

In [ ]:
person_df.columns

In [ ]:
# bin the continuous variables (income and age)
d = person_df.hh_income.describe()
bins = [0, d['25%'], d['50%'], d['75%'], d['max']]
labels = [0,1,2,3]
person_df['hh_income_bin'] = pd.cut(person_df.hh_income, bins=bins, labels=labels)

bins = [-1, 4, 13, 16, 21, 31, 41, 51, 61, 71, 81, 120]
labels = ['0-4', '4-13', '13-16','16-21', '21-31', '31-41', '41-51', '51-61', '61-71', '71-81', '81+']
# labels = [0, 4, 13, 16, 21, 31, 41, 51, 61, 71, 81]
person_df['age_bin'] = pd.cut(person_df.age, bins=bins, labels=labels)

person_df.hh_in_work = person_df.hh_in_work.map(header["hh_in_work"])
person_df.adult = person_df.adult.map(header["adult"])
person_df.gender = person_df.gender.map(header["gender"])
person_df.employment = person_df.employment.map(header["employment"])
person_df.occupation = person_df.occupation.map(header["occupation"])
person_df.main_activity = person_df.main_activity.map(header["main_activity"])
person_df["mode"] = person_df["mode"].map(header["mode"])
person_df.hh_income_bin = person_df.hh_income_bin.map(header["hh_income_bin"])

person_df.head()

#### Commuter Survey Output

We extract tours for work and education activities and create an zone based origin - destination freq table.

Note that we could further break-down by activity type and or mode choice.

In [ ]:
activities = ["work", "education"] # work & education
commuter_df = person_df.hh_index.loc[person_df.main_activity.isin(activities)]
commuter_freq_df = commuter_df.groupby([person_df.zone_id, person_df.main_facility_zone]).count()
commuter_freq_df.name = 'freq'
commuter_freq_df.to_csv(os.path.join(city_dir, 'zone_commuter_freq_table.csv'), header=True)

In [ ]:
# matrix version
commuter_freq_df.unstack().to_csv(os.path.join(city_dir, 'zone_commuter_freq_matrix.csv'), header=True)

In [ ]:
person_df.to_csv(os.path.join(city_dir, 'population_survey.csv'), header=True)

In [ ]:
person_df["mode"].describe()